In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sns.set()

In [2]:
raw_df = pd.read_csv('SCMS_Delivery_History_Dataset.csv')
raw_df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10324 entries, 0 to 10323
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            10324 non-null  int64  
 1   Project Code                  10324 non-null  object 
 2   PQ #                          10324 non-null  object 
 3   PO / SO #                     10324 non-null  object 
 4   ASN/DN #                      10324 non-null  object 
 5   Country                       10324 non-null  object 
 6   Managed By                    10324 non-null  object 
 7   Fulfill Via                   10324 non-null  object 
 8   Vendor INCO Term              10324 non-null  object 
 9   Shipment Mode                 9964 non-null   object 
 10  PQ First Sent to Client Date  10324 non-null  object 
 11  PO Sent to Vendor Date        10324 non-null  object 
 12  Scheduled Delivery Date       10324 non-null  object 
 13  D

In [21]:
def preprocessing_data(df, label_mapping):
    processed_data = df.copy()

    #Drop ID Column
    processed_data.drop('ID', axis=1, inplace=True)
    #Drop Missing target rows
    missing_target_rows = processed_data[processed_data['Shipment Mode'].isna()].index
    processed_data = processed_data.drop(missing_target_rows, axis=0).reset_index(drop=True)
    
    #Fill in missing values
    # Dosage with most frequent values
    # Line Item with mean()
    processed_data['Dosage'] = processed_data['Dosage'].fillna(processed_data['Dosage'].mode()[0])
    processed_data['Line Item Insurance (USD)'] = \
        processed_data['Line Item Insurance (USD)'].fillna(processed_data['Line Item Insurance (USD)'].mean())

    # Drop date columns with many missing values
    processed_data = processed_data.drop(columns=['PQ First Sent to Client Date','PO Sent to Vendor Date'], axis=1)

    # Extract date features
    for column in ['Scheduled Delivery Date','Delivered to Client Date','Delivery Recorded Date']:
        processed_data[column] = pd.to_datetime(processed_data[column])
        processed_data[column+' year'] = processed_data[column].apply(lambda x: x.year)
        processed_data.drop(column, axis=1, inplace=True)

    # Drop columns 'Weight (Kilograms)','Freight Cost (USD)' with > 90% missing values
    processed_data.drop(columns=['Weight (Kilograms)','Freight Cost (USD)'], axis=1, inplace=True) 

    # Drop high cardinality columns
    processed_data.drop(columns=['PQ #','PO / SO #','ASN/DN #'], axis= 1, inplace=True)

    # Binary Encoding
    processed_data['Fulfill Via']  = processed_data['Fulfill Via'].apply(lambda x: 1 if x == 'Direct Drop' else 0)
    processed_data['First Line Designation']  = \
        processed_data['First Line Designation'].apply(lambda x: 1 if x == 'Yes' else 0)

    # One hot encoding 
    for column in processed_data.drop('Shipment Mode', axis=1).select_dtypes('object').columns:
        dummies = pd.get_dummies(processed_data[column], prefix = column)
        processed_data = pd.concat([processed_data, dummies], axis=1)
        processed_data.drop(column, axis=1, inplace=True)
    
    # Split data
    y = processed_data['Shipment Mode']
    X = processed_data.drop('Shipment Mode', axis=1)

    #Encode teh labels
    y = y.replace(label_mapping)

    # Train test split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns = X_test.columns)

    return X_train, X_test, y_train, y_test

In [22]:
LABEL_MAPPING = {
    'Air':1,
    'Truck':2,
    'Air Charter':3,
    'Ocean':4
}
X_train, X_test, y_train, y_test = preprocessing_data(raw_df, label_mapping = LABEL_MAPPING)

In [23]:
X_train.head()

,Fulfill Via,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,First Line Designation,Line Item Insurance (USD),Scheduled Delivery Date year,Delivered to Client Date year,...,Manufacturing Site_Ranbaxy per Shasun Pharma Ltd,"Manufacturing Site_Ranbaxy, Paonta Shahib, India","Manufacturing Site_Remedica, Limassol, Cyprus",Manufacturing Site_Roche Basel,Manufacturing Site_Roche Madrid,"Manufacturing Site_Standard Diagnostics, Korea","Manufacturing Site_Strides, Bangalore, India.","Manufacturing Site_Trinity Biotech, Plc","Manufacturing Site_Weifa A.S., Hausmanngt. 6, P.O. Box 9113 GrÃ¸nland, 0133, Oslo, Norway",Manufacturing Site_bioLytical Laboratories
7995,-0.973396,-0.606820,0.054505,-0.177759,-0.413786,-0.198087,-1.464334,-0.284657,1.077039,1.077630,...,-0.011975,-0.052267,-0.011975,-0.065729,-0.035947,-0.10704,-0.238384,-0.206793,-0.011975,-0.023956
401,1.027331,0.291721,-0.449258,-0.401352,2.863044,0.364370,0.682904,0.004506,-1.825466,-1.813858,...,-0.011975,-0.052267,-0.011975,-0.065729,-0.035947,-0.10704,-0.238384,4.835764,-0.011975,-0.023956
5045,1.027331,1.318625,-0.429718,-0.319584,0.692473,-0.139119,-1.464334,-0.283076,-0.166892,-0.161579,...,-0.011975,-0.052267,-0.011975,-0.065729,-0.035947,-0.10704,-0.238384,-0.206793,-0.011975,-0.023956
9030,-0.973396,-0.606820,0.633631,0.129201,-0.414762,-0.198087,0.682904,0.406029,-0.581535,-0.574649,...,-0.011975,-0.052267,-0.011975,-0.065729,-0.035947,-0.10704,4.194918,-0.206793,-0.011975,-0.023956
4539,1.027331,2.088803,-0.402309,-0.442043,-0.490122,-0.261590,0.682904,-0.476847,1.077039,1.077630,...,-0.011975,-0.052267,-0.011975,-0.065729,-0.035947,-0.10704,-0.238384,-0.206793,-0.011975,-0.023956


# Training

In [ ]:
inputs = tf.keras.Input(X_train.shape[1])

In [ ]:

}

In [ ]:
for column in ['Weight (Kilograms)','Freight Cost (USD)']:
    print(processed_df[column].apply(lambda x: np.NaN if not x.isnumeric() else x).isna().mean())